In [6]:
# JS UseCase - https://observablehq.com/@cns-iu/ccf-api-usecase-javascript
import ccf_openapi_client

In [7]:
from ccf_openapi_client.api import default_api
from ccf_openapi_client.model.aggregate_count import AggregateCount
from ccf_openapi_client.model.database_status import DatabaseStatus
from ccf_openapi_client.model.error_message import ErrorMessage
from ccf_openapi_client.model.flat_spatial_placement import FlatSpatialPlacement
from ccf_openapi_client.model.get_spatial_placement_request import GetSpatialPlacementRequest
from ccf_openapi_client.model.min_max import MinMax
from ccf_openapi_client.model.ontology_tree import OntologyTree
from ccf_openapi_client.model.sparql_query_request import SparqlQueryRequest
from ccf_openapi_client.model.spatial_entity import SpatialEntity
from ccf_openapi_client.model.spatial_scene_node import SpatialSceneNode
from ccf_openapi_client.model.spatial_search import SpatialSearch
from ccf_openapi_client.model.tissue_block import TissueBlock
from pprint import pprint

In [8]:
configuration = ccf_openapi_client.Configuration(
    host = "https://ccf-api.hubmapconsortium.org/v1"
)
api_client = ccf_openapi_client.ApiClient(configuration)

api_instance = default_api.DefaultApi(api_client)

In [9]:
db_ready = False
result = None
while not db_ready:
    result = api_instance.db_status()
    if result['status'] == 'Ready':
        db_ready = True
    else:
        print('Database not ready yet! Retrying...', result)
        time.sleep(2)
print('Database ready!\n', result)

Database ready!
 {'checkback': 3600000,
 'load_time': 8768,
 'message': 'Database successfully loaded',
 'status': 'Ready'}


In [10]:
sex = "both"
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0002113",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",]
sceneResult = None
try:
    sceneResult = api_instance.scene(sex=sex, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms)
except ccf_openapi_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

In [11]:
sex = "both"
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0002113",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",]
tissueBlockResult = None
try:
    tissueBlockResult = api_instance.tissue_blocks(sex=sex, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms)
except ccf_openapi_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

In [12]:
sceneEntityASDict = {}
for scene in sceneResult:
    if 'entity_id' in scene:
        if scene['entity_id'] not in sceneEntityASDict:
            sceneEntityASDict[scene['entity_id']] = []
        sceneEntityASDict[scene['entity_id']].extend(scene['ccf_annotations'])

sceneEntityASDict

{'http://dx.doi.org/10.1016/j.trsl.2017.07.006#TissueBlock': ['http://purl.obolibrary.org/obo/UBERON_0002113',
  'http://purl.obolibrary.org/obo/UBERON_0013702',
  'http://purl.obolibrary.org/obo/UBERON_0004539'],
 'http://dx.doi.org/10.1681/ASN.2016091027#TissueBlockA': ['http://purl.obolibrary.org/obo/UBERON_0002113',
  'http://purl.obolibrary.org/obo/UBERON_0013702',
  'http://purl.obolibrary.org/obo/UBERON_0004539'],
 'http://dx.doi.org/10.1681/ASN.2016091027#TissueBlockB': ['http://purl.obolibrary.org/obo/UBERON_0002113',
  'http://purl.obolibrary.org/obo/UBERON_0013702',
  'http://purl.obolibrary.org/obo/UBERON_0004539'],
 'https://gtexportal.org/home/tissue/Kidney_Cortex#MTissueBlocks': ['http://purl.obolibrary.org/obo/UBERON_0002113',
  'http://purl.obolibrary.org/obo/UBERON_0001225',
  'http://purl.obolibrary.org/obo/UBERON_0001284',
  'http://purl.obolibrary.org/obo/UBERON_0002189',
  'http://purl.obolibrary.org/obo/UBERON_0000362',
  'http://purl.obolibrary.org/obo/UBERON_00

In [13]:
tDict = {}
for t in tissueBlockResult:
    tDict[t['sample_type']] = tDict.get(t['sample_type'], 0) + 1
tDict

{'Tissue Block': 94}

In [40]:
tissueBlockResult

[{'@id': 'http://dx.doi.org/10.1016/j.trsl.2017.07.006#TissueBlock',
  '@type': 'Sample',
  'datasets': [{'@id': 'http://dx.doi.org/10.1016/j.trsl.2017.07.006#Dataset',
                '@type': 'Dataset',
                'description': 'Data/Assay Types: Cytometry',
                'label': 'Registered 5/18/2020, Seth Winfree, KPMP-IU/OSU',
                'link': 'http://dx.doi.org/10.1016/j.trsl.2017.07.006',
                'technology': 'OTHER',
                'thumbnail': 'assets/kpmp/thumbnails/kpmp-sample.jpg'}],
  'description': '3.44 x 7.78 x 0.07 millimeter, 0.07 millimeter, nephrectomy, '
                 '0 Sections',
  'donor': {'@id': 'http://dx.doi.org/10.1016/j.trsl.2017.07.006#Donor',
            '@type': 'Donor',
            'description': 'Entered 5/18/2020, Seth Winfree, KPMP-IU/OSU',
            'label': 'CoverNephrectomy',
            'link': 'http://dx.doi.org/10.1016/j.trsl.2017.07.006',
            'providerName': 'KPMP-IU/OSU'},
  'label': 'Registered 5/18/20

In [18]:
query = '''PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ccf: <http://purl.org/ccf/>

SELECT DISTINCT (STR(?asLabel) as ?as_label) (STR(?qlabel) as ?cell_label) ?as_iri ?cell_iri WHERE {
  ?cell_iri ccf:ccf_located_in ?as_iri .
  ?cell_iri rdfs:label ?qlabel .
  ?as_iri rdfs:label ?asLabel .

  FILTER (?as_iri in (%s))
}'''

purlIris = set()

for k in sceneEntityASDict.keys():
    for v in sceneEntityASDict[k]:
        purlIris.add(v)
        

purlString = ", ".join("<" + s + ">" for s in purlIris)
queryResponse = None
try:
    queryResponse = api_instance.sparql(query=query % purlString, format='application/json')
except ccf_openapi_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

In [43]:
def reverseObject(input_object):
    reversed_object = {}

    for key, values in input_object.items():
        for value in values:
            if value not in reversed_object:
                reversed_object[value] = set()
            reversed_object[value].add(key)
    return reversed_object

ASEntityDict = reverseObject(sceneEntityASDict)
ASEntityDict

{'http://purl.obolibrary.org/obo/UBERON_0002113': {'http://dx.doi.org/10.1016/j.trsl.2017.07.006#TissueBlock',
  'http://dx.doi.org/10.1681/ASN.2016091027#TissueBlockA',
  'http://dx.doi.org/10.1681/ASN.2016091027#TissueBlockB',
  'https://entity.api.hubmapconsortium.org/entities/015f9976f3fed4c35dbbaad622a413d9',
  'https://entity.api.hubmapconsortium.org/entities/07c995983aac113b442c0dc0c0b42ae5',
  'https://entity.api.hubmapconsortium.org/entities/0a558711c6b9d5b9029717d722a2c31c',
  'https://entity.api.hubmapconsortium.org/entities/0be6dee31365fd1e590a45263df043c6',
  'https://entity.api.hubmapconsortium.org/entities/0ec930e0f4a8fcf12ea136a28e1acad4',
  'https://entity.api.hubmapconsortium.org/entities/11d794402e067e54888bb8a4af4184e0',
  'https://entity.api.hubmapconsortium.org/entities/221340e997dc0f2bf94e735098b148fe',
  'https://entity.api.hubmapconsortium.org/entities/2256c64aacd92af7028ffb33e478cf46',
  'https://entity.api.hubmapconsortium.org/entities/243c742cc28978c406d6509

In [44]:
#Can't use set because - unhashable type: 'dict'
mergedData = []
for response in queryResponse:
    entity_ids = ASEntityDict[response['as_iri']]
    for e in entity_ids:
        newResponse = response.copy()
        newResponse['block_id'] = e
        if newResponse not in mergedData:
            mergedData.append(newResponse)

Sheet with BlockId, AS, AS_LAbel, CT, CT_label

In [45]:
import csv
def save_to_csv(data, headers, filename):
     with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)

        # Write the headers
        writer.writerow(headers)

        # Write the data
        for obj in data:
            row = [obj.get(headers[header], '') for header in headers]
            writer.writerow(row)

header_mapping = {
    'Block Id': 'block_id',
    'AS': 'as_iri',
    'AS Label': 'as_label',
    'CT': 'cell_iri',
    'CT Label': 'cell_label'
}
save_to_csv(mergedData, header_mapping, 'output.csv')

In [ ]:
### Viz and Summary